In [1]:
from data.create_dataloaders import create_dataloaders
from model.smooth_cross_entropy import smooth_crossentropy
from model.wide_res_net import WideResNet
import torch

from train import OptimizerType, train_multiple_models

batch_size = 128
threads = 2

depth = 28
width_factor = 10
dropout = 0.0

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model = WideResNet(depth, width_factor, dropout, in_channels=3, labels=10).to(device)
train_dataloder, val_dataloader, test_dataloader = create_dataloaders(dataset_name="cifar10",
                                                                      batch_size=batch_size,
                                                                      num_workers=threads)

num_epochs = 2

configs = [
    {"model": model, "criterion": smooth_crossentropy, "optimizer": {"optimizer_type": OptimizerType.SGD}, "num_epochs": num_epochs, "model_name": "WRN-SGD"},
    {"model": model, "criterion": smooth_crossentropy, "optimizer": {"optimizer_type": OptimizerType.SAM}, "num_epochs": int(num_epochs / 2), "model_name": "WRN-SAM"},
    {"model": model, "criterion": smooth_crossentropy, "optimizer": {"optimizer_type": OptimizerType.ADAM}, "num_epochs": num_epochs, "model_name": "WRN-ADAM"},
]

train_multiple_models(configs, train_dataloder, val_dataloader, test_dataloader, "cifar10", device=device)

wandb: Currently logged in as: jhostic (jhostic-fer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Training model: WRN-SGD

Epoch 1/2


Training:  72%|███████▏  | 252/352 [02:56<01:09,  1.43it/s, batch_loss=1.5398, batch_acc=0.2578, avg_loss=1.6814]


KeyboardInterrupt: 